# Лабораторная работа №7.

# Задание №1.

In [1]:
class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def peek(self):
        return self.items[-1]

    def size(self):
        return len(self.items)

class BinaryTree:
    def __init__(self, rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None

    def insertLeft(self, newNode):
        if self.leftChild == None:
            self.leftChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.leftChild = self.leftChild
            self.leftChild = t

    def insertRight(self, newNode):
        if self.rightChild == None:
            self.rightChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.rightChild = self.rightChild
            self.rightChild = t

    def getRightChild(self):
        return self.rightChild

    def getLeftChild(self):
        return self.leftChild

    def setRootVal(self, obj):
        self.key = obj

    def getRootVal(self):
        return self.key

def buildParseTree(fpexp): # задание №1(a)
    fplist = fpexp.split()
    pStack = Stack()
    eTree = BinaryTree('')
    pStack.push(eTree)
    currentTree = eTree
    z = 0
    for i in fplist:
        if i == '(':
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        elif i not in ['&', '|', '-', ')']:
            currentTree.setRootVal(bool(int(i)))
            parent = pStack.pop()
            currentTree = parent
        elif i in ['&', '|']:
            currentTree.setRootVal(i)
            currentTree.insertRight('')
            pStack.push(currentTree)
            currentTree = currentTree.getRightChild()
        elif i == '-':
            parent = pStack.pop()
            currentTree = parent
            currentTree.setRootVal(i)
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        elif i == ')':
            currentTree = pStack.pop()
        else:
            raise ValueError
    return eTree

def evaluate(parseTree):
    opers = {'&': lambda x, y: x and y, '|': lambda x, y: x or y, '-': lambda x: not(x)}

    leftC = parseTree.getLeftChild()
    rightC = parseTree.getRightChild()
    if leftC and rightC:
        print(leftC.getRootVal(), " ", rightC.getRootVal())
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC), evaluate(rightC))
    elif leftC:
        print(leftC.getRootVal())
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC))
    else:
        return parseTree.getRootVal()

def printexp(tree): #задание №1(b)
    sVal = ""
    if tree:
        sVal = '( ' + printexp(tree.getLeftChild())
        sVal = sVal + ' ' + str(tree.getRootVal())
        sVal = sVal + ' ' + printexp(tree.getRightChild()) + ' )'
    return sVal


pt = buildParseTree("( ( 0 & 10 ) | ( - 1 ) )")
#pt = buildParseTree("( ( 1 & 0 ) | ( - 0 ) )") # при выводе будет True, False, False, True
#pt = buildParseTree("( ( 10 & 10 ) | ( - 10 ) )") # при выводе будет True, True, True, True
print(evaluate(pt))

print(printexp(pt))


&   -
False   True
True
False
( ( (  False  ) & (  True  ) ) | ( (  True  ) -  ) )


# Задание №2.

In [2]:
class TreeNode:
    def __init__(self, key, val, left=None, right=None, parent=None):
        self.key = key
        self.payload = val
        self.leftChild = left
        self.rightChild = right
        self.parent = parent

    def hasLeftChild(self):
        return self.leftChild

    def hasRightChild(self):
        return self.rightChild

    def isLeftChild(self):
        return self.parent and self.parent.leftChild == self

    def isRightChild(self):
        return self.parent and self.parent.rightChild == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.rightChild or self.leftChild)

    def hasAnyChildren(self):
        return self.rightChild or self.leftChild

    def hasBothChildren(self):
        return self.rightChild and self.leftChild

    def replaceNodeData(self, key, value, lc, rc):
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        if self.hasLeftChild():
            self.leftChild.parent = self
        if self.hasRightChild():
            self.rightChild.parent = self        

class BinarySearchTree(TreeNode):

    def __init__(self):
        self.root = None
        self.size = 0

    def length(self):
        return self.size

    def __len__(self):
        return self.size

    def put(self, key, val):
        if self.root:
            self._put(key, val, self.root)
        else:
            self.root = TreeNode(key, val)
        self.size = self.size + 1
    

    def _put(self, key, val, currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                self._put(key, val, currentNode.leftChild)
            else:
                currentNode.leftChild = TreeNode(key, val, parent=currentNode)
        else:
            if currentNode.hasRightChild():
                self._put(key, val, currentNode.rightChild)
            else:
                currentNode.rightChild = TreeNode(key, val, parent=currentNode)

    def __setitem__(self, k, v):
        self.put(k, v)

    def get(self, key):
        if self.root:
            res = self._get(key, self.root)
            if res:
                return res.payload
            else:
                return None
        else:
            return None

    def _get(self, key, currentNode):
        if not currentNode:
            return None
#             raise IndexError 
        elif currentNode.key == key:
            return currentNode
        elif key < currentNode.key:
            return self._get(key, currentNode.leftChild)
        else:
            return self._get(key, currentNode.rightChild)

    def __getitem__(self, key):
        return self.get(key)

    def __contains__(self, key):
        if self._get(key, self.root):
            return True
        else:
            return False

    def delete(self, key):
        if self.size > 1:
            nodeToRemove = self._get(key, self.root)
            if nodeToRemove:
                self.remove(nodeToRemove)
                self.size = self.size - 1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')

    def __delitem__(self, key):
        self.delete(key)

    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                self.parent.leftChild = None
            else:
                self.parent.rightChild = None
        elif self.hasAnyChildren():
            if self.hasLeftChild():
                if self.isLeftChild():
                    self.parent.leftChild = self.leftChild
                else:
                    self.parent.rightChild = self.leftChild
                self.leftChild.parent = self.parent
            else:
                if self.isLeftChild():
                    self.parent.leftChild = self.rightChild
                else:
                    self.parent.rightChild = self.rightChild
                self.rightChild.parent = self.parent

    def findSuccessor(self):
        succ = None
        if self.hasRightChild():
            succ = self.rightChild.findMin()
        else:
            if self.parent:
                if self.isLeftChild():
                    succ = self.parent
                else:
                    self.parent.rightChild = None
                    succ = self.parent.findSuccessor()
                    self.parent.rightChild = self
        return succ
    
    def findAllKeys(self):
        current_left = self
        current_right = self
        childrens = []
        while current_left.hasLeftChild:
            current_left = current_left.leftChild
            childrens.append(current_left)
        while current_right.hasLeftChild():
            current_right = current_right.rightChild
            childrens.append(current_right)            
        return childrens
    
    def findMin(self):
        current = self
        while current.hasLeftChild():
            current = current.leftChild
        return current

    def remove(self, currentNode):
        if currentNode.isLeaf():  
            if currentNode == currentNode.parent.leftChild:
                currentNode.parent.leftChild = None
            else:
                currentNode.parent.rightChild = None
        elif currentNode.hasBothChildren():  
            succ = currentNode.findSuccessor()
            succ.spliceOut()
            currentNode.key = succ.key
            currentNode.payload = succ.payload

        else:  
            if currentNode.hasLeftChild():
                if currentNode.isLeftChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.leftChild
                elif currentNode.isRightChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.leftChild
                else:
                    currentNode.replaceNodeData(currentNode.leftChild.key,
                                                currentNode.leftChild.payload,
                                                currentNode.leftChild.leftChild,
                                                currentNode.leftChild.rightChild)
            else:
                if currentNode.isLeftChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.rightChild
                elif currentNode.isRightChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.rightChild
                else:
                    currentNode.replaceNodeData(currentNode.rightChild.key,
                                                currentNode.rightChild.payload,
                                                currentNode.rightChild.leftChild,
                                                currentNode.rightChild.rightChild)

mytree = BinarySearchTree()
mytree[4] = "blue"
mytree[3] = "yellow"
mytree[2] = "red"
mytree[1] = "or"
print(mytree[4])
print(mytree[1]) 
mytree[5] = "green"
print(mytree[5])


blue
or
green


# Задание №3.

In [3]:
class BinaryTree:
    def __init__(self, rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None

    def insertLeft(self, newNode):
        if self.leftChild == None:
            self.leftChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.leftChild = self.leftChild
            self.leftChild = t

    def insertRight(self, newNode):
        if self.rightChild == None:
            self.rightChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.rightChild = self.rightChild
            self.rightChild = t

    def getRightChild(self):
        return self.rightChild

    def getLeftChild(self):
        return self.leftChild

    def setRootVal(self, obj):
        self.key = obj

    def getRootVal(self):
        return self.key

def sr(tree):
    if tree.getLeftChild() == tree.getRightChild() == None:
        print(f"Это {tree.getRootVal()}?")
        if input() == 'да':
            pass
        else:
            z = input(f"Какое животное вы загадали?\n")
            c = input(f"Какой вопрос поможет отличить {tree.getRootVal()} и {z}?\n")
            tree.insertLeft(z)
            tree.insertRight(tree.getRootVal())
            tree.setRootVal(c)
    else:
        print(tree.getRootVal())
        if input() == 'да':
            sr(tree.getLeftChild())
        else:
            sr(tree.getRightChild())



r = BinaryTree('Это млекопитающее?')
r.insertRight('Оно покрыто чешуей?')
r.getRightChild().insertRight('Змея')
r.getRightChild().insertLeft('Рыба')
r.insertLeft('Оно лает?')
r.getRightChild().insertRight('Птица')
r.getLeftChild()
r.getLeftChild().insertLeft('Собака')
r.getLeftChild().insertRight('Кошка')

In [4]:
print(sr(r))

Это млекопитающее?
да
Оно лает?
да
Это Собака?
нет
Какое животное вы загадали?
жираф
Какой вопрос поможет отличить Собака и жираф?
Какая шерсть
None


# Задание №4.

In [5]:
class Binstack:
    def __init__(self, maxsize):
        self.stackList = [0]
        self.currentsize = 0
        self.maxsize = maxsize
        
    def __str__(self):
        return str(self.stackList[1::])
    
    def __len__(self):
        return len(self.stackList[1::])
    def size(self):
        return self.currentsize

    def percUp(self, i):
        while i // 2 > 0:
            if self.stackList[i] < self.stackList[i // 2]:
                tmp = self.stackList[i // 2]
                self.stackList[i // 2] = self.stackList[i]
                self.stackList[i] = tmp
            i = i // 2

    def insert(self, k):
        if self.currentsize + 1 > self.maxsize:
            self.delMin()
            self.stackList.append(k)
            self.currentsize += 1
        else:     
            self.stackList.append(k)
            self.currentsize += 1
            self.percUp(self.currentsize)

    def percDown(self, i):
        while (i * 2) <= self.currentsize:
            mc = self.minChild(i)
            if self.stackList[i] > self.stackList[mc]:
                tmp = self.stackList[i]
                self.stackList[i] = self.stackList[mc]
                self.stackList[mc] = tmp
            i = mc

    def minChild(self, i):
        if i * 2 + 1 > self.currentsize:
            return i * 2
        else:
            if self.stackList[i * 2] < self.stackList[i * 2 + 1]:
                return i * 2
            else:
                return i * 2 + 1

    def delMin(self):
        retval = self.stackList[1]
        self.stackList[1] = self.stackList[self.currentsize]
        self.currentsize = self.currentsize - 1
        self.stackList.pop()
        self.percDown(1)
        return retval

    def buildstack(self, alist):
        i = len(alist) // 2
        self.currentsize = len(alist)
        self.stackList = [0] + alist[:]
        while (i > 0):
            self.percDown(i)
            i = i - 1

bh = Binstack(6)
bh.buildstack([9, 5, 6, 2, 3])

In [6]:
print('stack - ', bh)
print('len - ',len(bh))
print('size - ', bh.size())

print('='*90)

print('DEL MIN - ',bh.delMin())
print('stack - ',bh)
print('len - ',len(bh))
print('size - ',bh.size())

print('='*90)

print('INSERT 10')
bh.insert(10)
print('stack - ',bh)
print('len - ',len(bh))
print('size - ',bh.size())

print('='*90)

print('INSERT 20')
bh.insert(20)
print('stack - ',bh)
print('len - ',len(bh))
print('size - ',bh.size())

print('='*90)

print('INSERT 30')
bh.insert(30)
print('stack - ',bh)
print('len - ',len(bh))
print('size - ',bh.size())

print('='*90)

print('INSERT 11')
bh.insert(11)
print('stack - ',bh)
print('len - ',len(bh))
print('size - ',bh.size())

print('='*90)

print('INSERT 12')
bh.insert(12)
print('stack - ',bh)
print('len - ',len(bh))
print('size - ',bh.size())



print('='*90)
print('INSERT 13')
bh.insert(13)
print('stack - ',bh)
print('len - ',len(bh))
print('size - ',bh.size())

stack -  [2, 3, 6, 5, 9]
len -  5
size -  5
DEL MIN -  2
stack -  [3, 5, 6, 9]
len -  4
size -  4
INSERT 10
stack -  [3, 5, 6, 9, 10]
len -  5
size -  5
INSERT 20
stack -  [3, 5, 6, 9, 10, 20]
len -  6
size -  6
INSERT 30
stack -  [5, 9, 6, 20, 10, 30]
len -  6
size -  6
INSERT 11
stack -  [6, 9, 30, 20, 10, 11]
len -  6
size -  6
INSERT 12
stack -  [9, 10, 30, 20, 11, 12]
len -  6
size -  6
INSERT 13
stack -  [10, 11, 30, 20, 12, 13]
len -  6
size -  6


# Задание №5.

In [8]:
class Maxstack:
    def __init__(self, maxsize):
        self.stackList = [0]
        self.currentsize = 0
        self.maxsize = maxsize
        
    def __str__(self):
        return str(self.stackList[1::])
    
    def __len__(self):
        return len(self.stackList[1::])
    
    def __swap(self, i, j):
        self.stackList[i], self.stackList[j] = self.stackList[j], self.stackList[i]
        
    def size(self):
        return self.currentsize

    def percUp(self, index):
        parent = index//2
        if index <= 1:
            return
        elif self.stackList[index] > self.stackList[parent]:
            self.__swap(index, parent)
            self.percUp(parent)

    def insert(self, k):
        if self.currentsize + 1 > self.maxsize:
            self.pop()
            self.stackList.append(k)
            self.currentsize += 1
            self.percUp(len(self.stackList) - 1)
        else:     
            self.stackList.append(k)
            self.currentsize += 1
            self.percUp(self.currentsize)

    def percDown(self, i):
        while (i * 2) <= self.currentsize:
            mc = self.minChild(i)
            if self.stackList[i] > self.stackList[mc]:
                tmp = self.stackList[i]
                self.stackList[i] = self.stackList[mc]
                self.stackList[mc] = tmp
            i = mc

    def minChild(self, i):
        if i * 2 + 1 > self.currentsize:
            return i * 2
        else:
            if self.stackList[i * 2] < self.stackList[i * 2 + 1]:
                return i * 2
            else:
                return i * 2 + 1

    def pop(self):
        if len(self.stackList) > 2:
            self.__swap(1, len(self.stackList) - 1)
            max = self.stackList.pop()
            self.__bubbleDown(1)
        elif len(self.stackList) == 2:
            max = self.stackList.pop()
        else: 
            max = False
        return max
    
    def __bubbleDown(self, index):
        left = index * 2
        right = index * 2 + 1
        largest = index
        if len(self.stackList) > left and self.stackList[largest] < self.stackList[left]:
            largest = left
        if len(self.stackList) > right and self.stackList[largest] < self.stackList[right]:
            largest = right
        if largest != index:
            self.__swap(index, largest)
            self.__bubbleDown(largest)    

    def buildstack(self, alist):
        i = len(alist) // 2
        self.currentsize = len(alist)
        self.stackList = [0] + alist[:]
        while (i > 0):
            self.percDown(i)
            i = i - 1

bh = Maxstack(6)
bh.buildstack([9, 5, 6, 2, 3])


In [9]:
print(bh)

print('stack - ', bh)


print('='*90)


print('INSERT 10')
bh.insert(10)
print('stack - ',bh)


print('='*90)

print('INSERT 20')
bh.insert(20)
print('stack - ',bh)


print('='*90)

print('INSERT 30')
bh.insert(30)
print('stack - ',bh)


print('='*90)

print('INSERT 11')
bh.insert(11)
print('stack - ',bh)


print('='*90)

print('INSERT 12')
bh.insert(12)
print('stack - ',bh)




print('='*90)
print('INSERT 13')
bh.insert(13)
print('stack - ',bh)

print('='*90)
print('INSERT 8')
bh.insert(8)
print('stack - ',bh)

[2, 3, 6, 5, 9]
stack -  [2, 3, 6, 5, 9]
INSERT 10
stack -  [10, 3, 2, 5, 9, 6]
INSERT 20
stack -  [20, 3, 6, 5, 9, 2]
INSERT 30
stack -  [30, 3, 6, 5, 9, 2]
INSERT 11
stack -  [11, 3, 6, 5, 9, 2]
INSERT 12
stack -  [12, 3, 6, 5, 9, 2]
INSERT 13
stack -  [13, 3, 6, 5, 9, 2]
INSERT 8
stack -  [8, 3, 6, 5, 9, 2]


# Задание №6.

In [13]:
class Binstack:
    def __init__(self):
        self.stackList = [0]
        self.currentsize = 0
        
    def __str__(self):
        return str(self.stackList[1::])
    
    def __len__(self):
        return len(self.stackList[1::])  
    
    def percUp(self, i):
        while i // 2 > 0:
            if self.stackList[i] < self.stackList[i // 2]:
                tmp = self.stackList[i // 2]
                self.stackList[i // 2] = self.stackList[i]
                self.stackList[i] = tmp
            i = i // 2

    def insert(self, k):
        self.stackList.append(k)
        self.currentsize = self.currentsize + 1
        self.percUp(self.currentsize)

    def percDown(self, i):
        while (i * 2) <= self.currentsize:
            mc = self.minChild(i)
            if self.stackList[i] > self.stackList[mc]:
                tmp = self.stackList[i]
                self.stackList[i] = self.stackList[mc]
                self.stackList[mc] = tmp
            i = mc

    def minChild(self, i):
        if i * 2 + 1 > self.currentsize:
            return i * 2
        else:
            if self.stackList[i * 2] < self.stackList[i * 2 + 1]:
                return i * 2
            else:
                return i * 2 + 1

    def delMin(self):
        retval = self.stackList[1]
        self.stackList[1] = self.stackList[self.currentsize]
        self.currentsize = self.currentsize - 1
        self.stackList.pop()
        self.percDown(1)
        return retval

    def buildstack(self, alist):
        i = len(alist) // 2
        self.currentsize = len(alist)
        self.stackList = [0] + alist[:]
        while (i > 0):
            self.percDown(i)
            i = i - 1


bh = Binstack()
bh.buildstack([9, 5, 6, 2, 3])
print(bh)

class PriorityQueue(Binstack):

    def __init__(self):
        self.stack = Binstack()      
    def __str__(self):
        return str(self.stack)
    
    def enqueue(self, item, priority):
        self.stack.insert((item, priority))

    def dequeue(self):
        if self.stack is None:
            raise ValueError("This queue is empty.")
        self.stack.delMin()

    def size(self):
        return len(self.stack)

test = PriorityQueue()
test.enqueue(1, 2)
test.enqueue(1, 1)
test.enqueue(1, 2)
test.enqueue(3, 10)
print(test)
print(test.size())
test.dequeue()
print(test)
print(test.size())

[2, 3, 6, 5, 9]
[(1, 1), (1, 2), (1, 2), (3, 10)]
4
[(1, 2), (1, 2), (3, 10)]
3
